This file assesses a trade strategy for a stock by computing its winning percentage and profit-loss-ratio.

In [1]:
#import necessary packages
from jqdatasdk import *
auth("18639879991","Mzy1208@3")

auth success 


In [2]:
#return the d_values and closing prices of a stock over past 2 years
#@param scu name of the stock
#@param start_date start date of the 2 year period
#@param end_date end date of the 2 year period 
#@return a 2-tuple containing the d values list and closing price list
def get_DValue(scu,start_date,end_date):
    df=get_price(security=scu, start_date=start_date, end_date=end_date, 
        frequency='daily', fields=None, skip_paused=False, fq='pre', count=None)
    
    lowList = df['low'].rolling(9).min()
    lowList.fillna(value=df['low'].expanding().min(), inplace=True)
    highList = df['high'].rolling(9).max() 
    highList.fillna(value=df['high'].expanding().max(), inplace=True)
    rsv = (df.loc[:, 'close'] - lowList) / (highList - lowList) * 100
    df.loc[:, 'kdj_k'] = rsv.ewm(com=2).mean()
    df.loc[:, 'kdj_d'] = df.loc[:, 'kdj_k'].ewm(com=2).mean()
    df.loc[:, 'kdj_j'] = 3.0 * df.loc[:, 'kdj_k'] - 2.0 * df.loc[:, 'kdj_d']
    d_list=df['kdj_d'].tolist()
    close_price=df['close'].tolist()
    return d_list,close_price


In [6]:
#return the buy date index and sell date index in the list based on the trading strategy
#@param d_list the d-value list of a stock
#@return a 2-tuple containg the buy date index list and sell date index list
def compute_dates(d_list):
    length=len(d_list)
    i=0
    buy_index=[]
    sell_index=[]
    while(i<length):
        if d_list[i]<20:
            j=1
            while i+j<length-1 and j<8 and d_list[i+j]<80 :
                j=j+1
            sell_index.append(i+j)
            buy_index.append(i)
            i=i+j+1
            continue
        i=i+1
    return buy_index,sell_index

In [8]:
#compute and return the winning rate and profit-loss ratio of a stock
#@param buy_index the list containing all buy date indices 
#@param sell_index the list containing all sell date indices
#@param close_price the list containing all closing prices 
#@return a 2-tuple containing the winning rate list and profit-loss ratio list
def compute_winning_rate_and_profit_loss_ratio(buy_index,sell_index,close_price):
    buy_price=[]
    sell_price=[]
    profit_list=[]
    num_trade=len(buy_index)
    win_num=0
    total_gain=0
    total_loss=0
    for i in range(num_trade):
        buy_price.append(close_price[buy_index[i]])
        sell_price.append(close_price[sell_index[i]])
        profit_list.append(sell_price[i]-buy_price[i])
        if profit_list[i]>0:
            win_num+=1
            total_gain+=profit_list[i]
        else:
            total_loss+=abs(profit_list[i])
        
    win_rate=win_num/num_trade 
    profit_loss_ratio=(total_gain/win_num)/(total_loss/(num_trade-win_num))
    return win_rate,profit_loss_ratio

In [ ]:
#compute the winning rate and profit-loss ratio of the stock 000001.XHSE from 2020-01-01 to 2021-12-31
(d_list,close_price)=get_DValue("000001.XSHE","2020-01-01","2021-12-31")
(buy_index,sell_index)=compute_dates(d_list)
(win_rate,profit_loss_ratio)=compute_winning_rate_and_profit_loss_ratio(buy_index,sell_index,close_price)
print(win_rate)
print(profit_loss_ratio)

In [ ]:
#log out the joinquant account
logout()